In [1]:
import pickle
from typing import Tuple, List
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import SGDClassifier
from scribe_classifier.data.canada.NOCdb.models.simple_model import SimpleModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from scribe_classifier.data.canada.NOCdb.readers import TitleSet, TitleRecord
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from scipy import sparse
from sklearn.preprocessing import LabelEncoder
from scribe_classifier.data.canada.NOCdb.readers import AllCodes
from scribe_classifier.data.canada.NOCdb.models.neural_networks.artificial_neural_net import ANNclassifier
# from keras.layers import Dense, Dropout
# from keras.models import Sequential, load_model
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from scribe_classifier.data.canada.NOCdb.models.neural_networks.combined_models import CombinedModels

Using TensorFlow backend.


In [2]:
mdl_strs = dict()
for target_level in range(1,4):
    level_mdl_strs = dict()
    level_mdl_strs['sgd'] = 'source_data/pickles/canada/trained_models/simple.lvl%d.sgdsv.P' % target_level
    level_mdl_strs['bayes'] = 'source_data/pickles/canada/trained_models/simple.lvl%d.bayes.P' % target_level
    level_mdl_strs['ann'] = 'nnmodels/ANN/neural_net_level%d.P' % target_level
    mdl_strs[target_level]= level_mdl_strs

In [3]:
target_level=1

NameError: name 'lbl_bin' is not defined

In [ ]:
cmb_mdls = CombinedModels('source_data/pickles/canada/tidy_sets/all_codes.P',
                          mdl_strs[1],
                          mdl_strs[2],
                          mdl_strs[3],
                          target_level=target_level)

In [ ]:
train = TitleSet.load_from_pickle('source_data/pickles/canada/test_sets/train.set.lvl%d.P' % target_level, is_path=True).copy_and_append_empty_string_class()
test = TitleSet.load_from_pickle('source_data/pickles/canada/test_sets/test.set.lvl%d.P' % target_level, is_path=True).copy_and_append_empty_string_class()
valid = TitleSet.load_from_pickle('source_data/pickles/canada/test_sets/valid.set.lvl%d.P' % target_level, is_path=True).copy_and_append_empty_string_class()

In [ ]:
T_titles = train.get_title_vec()
t_titles = test.get_title_vec()
v_titles = valid.get_title_vec()
T_labels = train.get_code_vec(target_level=target_level)
t_labels = test.get_code_vec(target_level=target_level)
v_labels = valid.get_code_vec(target_level=target_level)


In [ ]:
t_preds = cmb_mdls.predict(t_titles)

In [ ]:
print(metrics.classification_report(t_labels, t_preds))
print(metrics.accuracy_score(t_labels, t_preds))

In [ ]:
T_preds = cmb_mdls.predict(T_titles)

In [ ]:
print(metrics.classification_report(T_labels, T_preds))
print(metrics.accuracy_score(T_labels, T_preds))

In [ ]:
cmb_mdls.predict(['farmer'])